In [1]:
import pandas as pd
import pandas_datareader as pdr
from bs4 import BeautifulSoup
import requests
import json
import re
import numpy as np
from nltk.corpus import stopwords

C:\Users\Lincoln\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


# prepare codemap

In [232]:
# 종목 타입에 따라 download url이 다름. 종목코드 뒤에 .KS .KQ등이 입력되어야해서 Download Link 구분 필요
stock_type = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt'
    }

# download url 조합
def get_download_stock(market_type=None):
    market_type = stock_type[market_type]
    download_link = 'http://kind.krx.co.kr/corpgeneral/corpList.do'
    download_link = download_link + '?method=download'
    download_link = download_link + '&marketType=' + market_type
    df = pd.read_html(download_link, header=0)[0]
    return df;
# kospi 종목코드 목록 다운로드
def get_download_kospi():
    df = get_download_stock('kospi')
    df.종목코드 = df.종목코드.map('{:06d}.KS'.format)
    return df
# kosdaq 종목코드 목록 다운로드
def get_download_kosdaq():
    df = get_download_stock('kosdaq')
    df.종목코드 = df.종목코드.map('{:06d}.KQ'.format)
    return df
# kospi, kosdaq 종목코드 각각 다운로드
kospi_df = get_download_kospi()
kosdaq_df = get_download_kosdaq()
# data frame merge
code_df = pd.concat([kospi_df, kosdaq_df])
# data frame정리
code_df = code_df[['회사명', '종목코드']]
# data frame title 변경 '회사명' = name, 종목코드 = 'code'
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

# read headlines(daum news)

In [156]:
p_dict = ['↑', '급상승', '상승', '올랐다', '오름세', '호재', '안착', '대박', '기회', '잡았나', '출발', '반등', '사자', '급등', '승부', '증가', '최고치', '안정적', '등재', 'buy', '개선']
n_dict = ['↓', '상장폐지', '뚝', '관리종목', '주의보', '낚시', '우려', '둔화', '공포', '사임', '어렵다', '어려운 ', '썰렁', '손실', '적자', '과징금', '하락', '약화', '애로']
stopword = ['이다', '너무', '은', '는', '이', '가', '을', '를', '에', '게', '도', '行', '"', "'", '.', '[', ']', ',']

In [266]:
def get_news(date_list):
    news = pd.DataFrame()
    for date in date_list:
        headline = []
        for page in range(1, 1000):
            url = f'https://news.daum.net/breakingnews/economic?page={page}&regDate={date}'
            html = requests.get(url).text
            soup = BeautifulSoup(html, 'html.parser')

            if len(soup.findAll("a", href=re.compile("https://v.daum.net/v/"))) is 0:
                break
            else:
                for index, link in enumerate(soup.findAll("a", href=re.compile("https://v.daum.net/v/"))):
                    if (link.text != None):
                        headline.append(link.text)

        headlines = set(list(headline))
        df = pd.DataFrame(headlines, columns=['headline'])
        df['date'] = date
        news = pd.concat([news, df])
    return news

In [157]:
def p_scoring(text):
    score = 0
    for keyword in p_dict:
        if keyword in text:
            score += 1
    return score

def n_scoring(text):
    score = 0
    for keyword in n_dict:
        if keyword in text:
            score += 1
    return score

In [159]:
def sw(text):
    for i in stopword:
        if i in text:
            text = text.replace(i, ' ')
    return text

In [269]:
# 회사명으로 주식 종목 코드를 획득할 수 있도록 하는 함수
def get_code(df, name):
    try:
        if len(df.query(f"name=='{name}'")['code']) == 0:
            return False
        else:
            code = df.query(f"name=='{name}'")['code'].to_string(index=False)
            # 위와같이 code명을 가져오면 앞에 공백이 붙어있는 상황이 발생하여 앞뒤로 sript() 하여 공백 제거
            code = code.strip()
            return code
    except:
        print(name)

In [270]:
def is_code(text):
    tokenization = text.split(' ')
    codes = []
    for token in tokenization:
        if len(token) != 0:
            if get_code(code_df, token.strip()):
                codes.append(get_code(code_df, token))
    if len(codes) == 0:
        return False
    else: return codes

In [267]:
news = get_news(['20200214', '20200215'])
news['p_score'] = news.headline.apply(p_scoring)
news['n_score'] = news.headline.apply(n_scoring)

In [268]:
news['text'] = news.headline.apply(sw)

In [274]:
news['is_code'] = news.text.apply(is_code)

In [275]:
news.query('is_code != False')

,headline,date,p_score,n_score,text,is_code
5,"[잠정실적]동일금속, 작년 4Q 매출액 179억(-24%) 영업이익 11.8억(-4...",20200214,0,0,잠정실적 동일금속 작년 4Q 매출액 179억(-24%) 영업 익 11 8억(-4...,[109860.KQ]
11,"웅진씽크빅, '웅진스마트올' 홈쇼핑 앵콜 방송",20200214,0,0,웅진씽크빅 웅진스마트올 홈쇼핑 앵콜 방송,[095720.KS]
12,'깜짝 배당' 코리안리..주주친화 행보 빛볼까,20200214,0,0,깜짝 배당 코리안리 주주친화 행보 빛볼까,[003690.KS]
13,"대한항공 직원들 KCGI·조현아 제안에 ""이런 사람들이 전문경영인?""",20200214,0,0,대한항공 직원들 KCGI·조현아 제안 런 사람들 전문경영인?,[003490.KS]
17,"'S&T모티브' 5% 이상 상승, 전일 기관 대량 순매수",20200214,1,0,S&T모티브 5% 상 상승 전일 기관 대량 순매수,[064960.KS]
...,...,...,...,...,...,...
398,"컴투스, 턴제RPG '히어로즈워: 카운터어택'..이미지 첫 공개",20200215,0,0,컴투스 턴제RPG 히어로즈워: 카운터어택 미지 첫 공개,[078340.KQ]
411,"[fn마켓워치]큐캐피탈, 카카오VX에 200억 투자",20200215,0,0,fn마켓워치 큐캐피탈 카카오VX 200억 투자,[016600.KQ]
426,"캐시카우 두산밥캣, 그룹 견인차 '우뚝'",20200215,0,1,캐시카우 두산밥캣 그룹 견인차 우뚝,[241560.KS]
440,"한섬, 클럽모나코 '오케이셔널 드레싱, 러브 클럽 모나코' 캡슐 컬렉션 출시",20200215,0,0,한섬 클럽모나코 오케 셔널 드레싱 러브 클럽 모나코 캡슐 컬렉션 출시,[020000.KS]


In [278]:
news.sample(10)

,headline,date,p_score,n_score,text,is_code
413,"[가상화폐] 15일 오후 4시 30분 비트코인 -12,000원 하락한 11,833,...",20200215,0,1,상화폐 15일 오후 4시 30분 비트코인 -12 000원 하락한 11 833 ...,False
3972,"한은, '코로나19'에 금융중개지원대출 카드 꺼내",20200214,0,0,한 코로나19 금융중개지원대출 카드 꺼내,False
3767,"[포토] ""경제 살리자"" 거시경제 금융회의",20200214,0,0,포토 경제 살리자 거시경제 금융회의,False
59,"'코로나19' 1월 마스크 소비자상담, 전월比 10배..주문취소 불만 폭주",20200214,0,0,코로나19 1월 마스크 소비자상담 전월比 10배 주문취소 불만 폭주,False
2834,"하나銀, 모바일 보증부대출 개발 나선다",20200214,0,0,하나銀 모바일 보증부대출 개발 나선다,False
4018,"웅진씽크빅, AI학습 '웅진스마트올' 앵콜 방송",20200214,0,0,웅진씽크빅 AI학습 웅진스마트올 앵콜 방송,[095720.KS]
1398,"신세계사이먼, 여주시 아동시설 물품지원을 위한 기부금 출연",20200214,0,0,신세계사 먼 여주시 아동시설 물품지원 위한 기부금 출연,False
2327,"맘스터치, 통감자X통새우 '쉬림프포테이토버거' 3종 라인업 갖춰",20200214,0,0,맘스터치 통감자X통새우 쉬림프포테 토버거 3종 라인업 갖춰,False
4021,이상훈 삼성전자 이사회 의장 사임.. 법정구속 여파,20200214,0,1,상훈 삼성전자 사회 의장 사임 법정구속 여파,[005930.KS]
3033,"부영그룹 우정교육문화재단, 2020년도 1학기 외국인 유학생 장학금 지급",20200214,0,0,부영그룹 우정교육문화재단 2020년 1학기 외국인 유학생 장학금 지급,False


In [281]:
df = pdr.get_data_yahoo('095720.KS', '2020-02-14', '2020-02-15')

In [282]:
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-02-14,2670,2635,2655,2660,430003,2660


In [284]:
import FinanceDataReader as fdr

df = fdr.DataReader('005930', '2020-02-14', '2020-02-15')
df.head(10)

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-02-14,60900,61900,60200,61800,13256502,0.018122
